In [13]:
%%writefile query.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE stackoverflow_;

DROP TABLE if exists posts_sample_external; 

CREATE EXTERNAL TABLE posts_sample_external 
(row_id string,
post_type_id string,
year string,
month string)
ROW FORMAT 
SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' 
WITH SERDEPROPERTIES (
    "input.regex" = ".*?(?=.*\\bId=\"(\\d+)\")(?=.*\\bPostTypeId=\"(\\d+)\")(?=.*\\bCreationDate=\"(\\d+)-(\\d+)).*$"
)
LOCATION '/data/stackexchange1000/posts';

Overwriting query.hql


In [14]:
%%writefile query2.hql

USE stackoverflow_;

DROP TABLE if exists posts_sample; 

CREATE TABLE posts_sample 
(count int) 
PARTITIONED BY (year string, month string);

Overwriting query2.hql


In [15]:
%%writefile query3.hql

set hive.exec.dynamic.partition.mode=nonstrict;

USE stackoverflow_;

FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT count(*) as count, year, concat(year,"-",month) as month
WHERE year IS NOT NULL
GROUP BY year, month;

Overwriting query3.hql


In [16]:
%%writefile query4.hql
USE stackoverflow_;
SELECT year, month, count FROM posts_sample where month='2008-10';

Overwriting query4.hql


In [17]:
! hive -S -f query.hql
! hive -S -f query2.hql
! hive -S -f query3.hql

In [18]:
! hive -f query4.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.142 seconds
OK
2008	2008-10	73
Time taken: 2.886 seconds, Fetched: 1 row(s)
